# Homework 12

https://scikit-learn.org/0.15/modules/scaling_strategies.html#incremental-learning

* Implement a mini batch functionality to train a regressor.
    - (Optional) If anyone want to do this in a pipeline can do this: https://koaning.github.io/tokenwiser/api/pipeline.html

* Save model, load the model again and test it on `X_test`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
def test_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)

    df = df.sample(5000, random_state=100).reset_index(drop=True)
    
    y = df['sellingprice']
    df.drop('sellingprice', axis=1, inplace=True)
    X = df
    
    return X,y

def partial_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)
   
    while(True):
        yield df.sample(100).reset_index(drop=True)
        
gen = partial_df()

In [3]:
X_test, y_test = test_df()

In [4]:
# each time you call this you will get a new slice of the dataframe.
next(gen)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2013,Chevrolet,Cruze,Eco,sedan,manual,1g1pj5sb6d7329654,nv,3.1,38123.0,blue,black,gm financial,10250,10100,Fri Jun 05 2015 05:00:00 GMT-0700 (PDT)
1,2011,Lincoln,Navigator,L,SUV,NaN,5lmjj3j50bej04922,pa,3.6,129429.0,black,black,sports car leasing,23400,19000,Fri Jan 23 2015 01:00:00 GMT-0800 (PST)
2,2013,Volkswagen,Passat,SE PZEV,Sedan,automatic,1vwbp7a37dc109515,tn,4.3,60678.0,burgundy,black,the hertz corporation,11400,11500,Wed Jan 21 2015 02:30:00 GMT-0800 (PST)
3,2012,Chevrolet,Malibu,LT,Sedan,automatic,1g1zc5e02cf132496,wi,3.6,32483.0,black,gray,gm financial leasing,11100,9800,Wed Jan 21 2015 02:00:00 GMT-0800 (PST)
4,2012,Ford,Fusion,SE,Sedan,automatic,3fahp0hg6cr291758,fl,2.9,62543.0,black,black,rental car finance corp/(pen),9975,9300,Thu May 21 2015 02:15:00 GMT-0700 (PDT)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2011,Cadillac,CTS,Luxury,sedan,NaN,1g6dg5ey9b0150204,pa,4.3,24268.0,—,tan,d l newsome automotive inc,20200,18500,Fri Jun 19 2015 02:15:00 GMT-0700 (PDT)
96,2012,Volkswagen,CC,R-Line,Sedan,manual,wvwnn7an1ce515588,fl,3.1,49648.0,black,black,tfp international inc,12650,11500,Thu Jan 15 2015 02:30:00 GMT-0800 (PST)
97,2005,Ford,F-150,FX4,SuperCab,automatic,1ftpx14585na14521,ma,3.4,146097.0,black,black,imperial chevrolet,6975,7100,Tue Mar 03 2015 02:00:00 GMT-0800 (PST)
98,2005,Mercury,Mountaineer,Premier,SUV,automatic,4m2du86w05uj16485,ne,2.8,184054.0,silver,tan,diers inc,1875,1850,Thu Feb 05 2015 03:00:00 GMT-0800 (PST)


In [66]:
df.columns


Index(['year', 'make', 'model', 'trim', 'body', 'transmission', 'vin', 'state',
       'condition', 'odometer', 'color', 'interior', 'seller', 'mmr',
       'sellingprice', 'saledate'],
      dtype='object')

In [85]:
def test_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)
    df = df.sample(5000, random_state=100).reset_index(drop=True)
    
    # Drop rows with non-numeric values in 'year', 'odometer', and 'mmr' columns
    df = df[df['year'].str.isnumeric()]
    df = df[df['odometer'].str.isnumeric()]
    df = df[df['mmr'].str.isnumeric()]
    
    y = df['sellingprice']
    df.drop('sellingprice', axis=1, inplace=True)
    X = df
    
    return X, y


In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import SGDRegressor

def mini_batch_training(X_train, y_train, batch_size):
    # Initialize the encoder, scaler, and regressor
    enc = OneHotEncoder(sparse=False)
    scaler = StandardScaler()
    regressor = SGDRegressor()

    # Fit the encoder on the entire training data
    enc.fit(X_train[['make']])

    # Fit the scaler on the numerical features of the training data
    scaler.fit(X_train[['year', 'odometer', 'mmr']])

    # Train the regressor using mini-batch
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]

        # Perform encoding, scaling, and concatenation for each batch
        X_encoded = enc.transform(X_batch[['make']])
        X_scaled = scaler.transform(X_batch[['year', 'odometer', 'mmr']])
        X_transformed = np.concatenate([X_encoded, X_scaled, X_batch[['trim', 'body', 'transmission', 'state', 'condition', 'color', 'interior']]], axis=1)

        # Perform partial fit on the regressor
        regressor.partial_fit(X_transformed, y_batch)

    return regressor


In [13]:
def evaluate_model(regressor, X_test, y_test):
    # Predict on the test data
    y_pred = regressor.predict(X_test)

   
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Squared Error:", mse)


In [14]:
def save_model(regressor, filename):
    # Save the trained model
    joblib.dump(regressor, filename)

def load_model(filename):
    # Load the saved model
    loaded_model = joblib.load(filename)
    return loaded_model


In [22]:
from sklearn.preprocessing import LabelEncoder

# Convert the make feature to a numerical feature
encoder = LabelEncoder()
df['make'] = encoder.fit_transform(df['make'])
